<a href="https://colab.research.google.com/github/Varunkumar2000-alt/Clustering-Project/blob/main/Optimize_CNN_model.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install keras-tuner

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 128.9/128.9 kB 2.7 MB/s eta 0:00:00


In [2]:
import tensorflow as tf
from tensorflow import keras
import numpy as np

In [3]:
print(tf.__version__)

2.15.0


In [4]:
fashion_mnist=keras.datasets.fashion_mnist

In [5]:
(train_images,train_labels),(test_images,test_labels)=fashion_mnist.load_data()

4422102/4422102 [==============================] - 0s 0us/step


In [6]:
train_images=train_images/255.0
test_images=test_images/255.0

In [7]:
train_images[0].shape

(28, 28)

In [8]:
train_images=train_images.reshape(len(train_images),28,28,1)
test_images=test_images.reshape(len(test_images),28,28,1)

In [9]:
def build_model(hp):
  model = keras.Sequential([
    keras.layers.Conv2D(
        filters=hp.Int('conv_1_filter', min_value=32, max_value=128, step=16),
        kernel_size=hp.Choice('conv_1_kernel', values = [3,5]),
        activation='relu',
        input_shape=(28,28,1)
    ),
    keras.layers.Conv2D(
        filters=hp.Int('conv_2_filter', min_value=32, max_value=64, step=16),
        kernel_size=hp.Choice('conv_2_kernel', values = [3,5]),
        activation='relu'
    ),
    keras.layers.Flatten(),
    keras.layers.Dense(
        units=hp.Int('dense_1_units', min_value=32, max_value=128, step=16),
        activation='relu'
    ),
    keras.layers.Dense(10, activation='softmax')
  ])

  model.compile(optimizer=keras.optimizers.Adam(hp.Choice('learning_rate', values=[1e-2, 1e-3])),
              loss='sparse_categorical_crossentropy',
              metrics=['accuracy'])

  return model

In [10]:
from kerastuner import RandomSearch
from kerastuner.engine.hyperparameters import HyperParameters

<ipython-input-10-2bbfbe7dae29>:1: DeprecationWarning: `import kerastuner` is deprecated, please use `import keras_tuner`.
  from kerastuner import RandomSearch


In [11]:
tuner_search=RandomSearch(build_model,
                          objective='val_accuracy',
                          max_trials=5,directory='output',project_name="Mnist Fashion")

In [12]:
tuner_search.search(train_images,train_labels,epochs=3,validation_split=0.1)

Trial 5 Complete [00h 00m 42s]
val_accuracy: 0.8526666760444641

Best val_accuracy So Far: 0.9146666526794434
Total elapsed time: 00h 02m 54s


In [13]:
model=tuner_search.get_best_models(num_models=1)[0]

In [14]:
model.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv2d (Conv2D)             (None, 26, 26, 128)       1280      
                                                                 
 conv2d_1 (Conv2D)           (None, 24, 24, 48)        55344     
                                                                 
 flatten (Flatten)           (None, 27648)             0         
                                                                 
 dense (Dense)               (None, 112)               3096688   
                                                                 
 dense_1 (Dense)             (None, 10)                1130      
                                                                 
Total params: 3154442 (12.03 MB)
Trainable params: 3154442 (12.03 MB)
Non-trainable params: 0 (0.00 Byte)
_________________________________________________________________


In [15]:
model.fit(train_images, train_labels, epochs=10, validation_split=0.1, initial_epoch=3)

Epoch 4/10
1688/1688 [==============================] - 11s 6ms/step - loss: 0.1180 - accuracy: 0.9551 - val_loss: 0.2638 - val_accuracy: 0.9135
Epoch 5/10
1688/1688 [==============================] - 10s 6ms/step - loss: 0.0784 - accuracy: 0.9705 - val_loss: 0.3239 - val_accuracy: 0.9090
Epoch 6/10
1688/1688 [==============================] - 9s 5ms/step - loss: 0.0508 - accuracy: 0.9811 - val_loss: 0.3767 - val_accuracy: 0.9143
Epoch 7/10
1688/1688 [==============================] - 10s 6ms/step - loss: 0.0375 - accuracy: 0.9864 - val_loss: 0.3989 - val_accuracy: 0.9133
Epoch 8/10
1688/1688 [==============================] - 10s 6ms/step - loss: 0.0285 - accuracy: 0.9905 - val_loss: 0.4370 - val_accuracy: 0.9107
Epoch 9/10
1688/1688 [==============================] - 10s 6ms/step - loss: 0.0218 - accuracy: 0.9923 - val_loss: 0.4893 - val_accuracy: 0.9110
Epoch 10/10
1688/1688 [==============================] - 9s 6ms/step - loss: 0.0211 - accuracy: 0.9923 - val_loss: 0.4944 - val_acc